# Diferenciación automática

## Introducción a los tipos

En este notebook vamos a empezar a ver qué son los tipos en Julia, para así construir un diferenciador. Un tipo en Julia es como una familia, o una 'especie' de variables, como lo son los "Float64", los complejos o los booleanos, entre otros.

Como ejemplo, vamos a construir un tipo que llamaremos "mi_vec", el cual será un vector. Tendremos que definir más adelante todas las operaciones entre este tipo de variables.

In [6]:
type mi_vec
    x
    y
end

In [8]:
?mi_vec

search:



No documentation found.

**Summary:**

```
type mi_vec <: Any
```

**Fields:**

```
x :: Any
y :: Any
```


Como podemos ver, 'mi_vec' es entonces un tipo que está compuesto de los campos 'x' y 'y', los cuales pueden ser de cualquier tipo a su vez.

In [9]:
A=mi_vec(1,2)

mi_vec(1,2)

Para acceder a los campos de este tipo, se utiliza la siguiente sintáxis:

In [11]:
A.x

1

In [13]:
A.y

2

Ahora, vamos a ver que pasa cuando intentamos hacer operaciones para este tipo.

In [14]:
A+A

LoadError: MethodError: no method matching +(::mi_vec, ::mi_vec)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138[0m

Como podemos ver, aún no se ha definido ninguna suma para este tipo, por lo cual Julia no sabe como sumar dos objetos del tipo 'mi_vec'. Para definir esto, primero hay que importar la función de suma, ya que Julia no nos permite hacer cambios en las funciones de Base (sean las funciones principales de Julia) si no las hemos importado.

In [16]:
import Base.+

In [17]:
+(A::mi_vec,B::mi_vec)=mi_vec(A.x+B.x , A.y+B.y)

+ (generic function with 164 methods)

Ahora sí, vamos a poder hacer la suma anterior:

In [22]:
A+A

mi_vec(2,4)

Si quisiéramos definir cómo se suman los objetos de este tipo con un real, o con cualquier otro tipo, vamos a tener que definir la suma para esos tipos entonces.

In [23]:
A+2

LoadError: MethodError: no method matching +(::mi_vec, ::Int64)[0m
Closest candidates are:
  +(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  +([1m[31m::Complex{Bool}[0m, ::Real) at complex.jl:151
  +([1m[31m::Char[0m, ::Integer) at char.jl:40
  ...[0m

In [24]:
2*A

LoadError: MethodError: no method matching *(::Int64, ::mi_vec)[0m
Closest candidates are:
  *(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m) at operators.jl:138
  *{T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}}(::T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}, [1m[31m::T<:Union{Int128,Int16,Int32,Int64,Int8,UInt128,UInt16,UInt32,UInt64,UInt8}[0m) at int.jl:33
  *(::Real, [1m[31m::Complex{Bool}[0m) at complex.jl:158
  ...[0m

Como se puede ver, tendríamos que definir todas las operaciones pertinentes para cada nuevo tipo que queramos construir. También se pueden definir tipos que tengan a los tipos de sus campos fijos.

In [27]:
type mi_vec2{T<:Real}
    x :: T
    y :: T
end

Este tipo sólo va a aceptar entradas para las cuales ambos campos sean reales entonces:

In [28]:
mi_vec2(1,2)

mi_vec2{Int64}(1,2)

In [29]:
mi_vec2(2,2.2)

LoadError: MethodError: no method matching mi_vec2{T<:Real}(::Int64, ::Float64)[0m
Closest candidates are:
  mi_vec2{T<:Real}{T<:Real}(::T<:Real, [1m[31m::T<:Real[0m) at In[27]:2
  mi_vec2{T<:Real}{T}(::Any) at sysimg.jl:53[0m

Como podemos ver, esta sintáxis obliga a que ambos campos tengan el mismo tipo. Se tendría que utilizar una sintáxis diferente para cambiar esto.

In [1]:
type mi_vec3{T1<: Number, T2<: Number}
    x :: T1
    y :: T2
end

In [2]:
mi_vec3(1,2)

mi_vec3{Int64,Int64}(1,2)

In [3]:
mi_vec3(1,2.2)

mi_vec3{Int64,Float64}(1,2.2)

Ojo, con Julia 0.6 la sintaxis va a cambiar. En vez de usar 'type' vamos a tener que usar 'mutable struct', y en vez de  '{T<:Integer}' (o cualquier otro tipo) vamos a tener que utilizar '{T} where T<:Integer'.
<p> Todos estos cambios se pueden ver con más detalle en https://github.com/JuliaLang/julia/blob/master/NEWS.md

## Construyendo el tipo dual

Para el diferenciador automático, vamos a construir el tipo 'Dual'. Este tipo va a tener dos campos. Sea $u(x)$ una función, y sea $\vec u(x_0)$ un elemento de este tipo, se tendrá $\vec u(x_0)=(u_0,u'_0)$, teniendo $u_0=u(x_0)$, y $u'_0=u'(x_0)$.
<p> Vamos a querer entonces que, sean $\vec u, \vec w \in \mathcal{D}$, se tengan las operaciones siguientes:
- $\vec u(x_0) \pm \vec w(x_0)=(u_0\pm w_0, u'_0 \pm w'_0)$ 
- $\vec u(x_0)*\vec w(x_0)=(u_0*w_0, w_0*u'_0+u_0*w'_0)$
- $\frac{\vec{u}(x_0)}{\vec{w}(x_0)}=(\frac{u_0}{w_0}, \frac{u'_0*w_0-u_0*w'_0}{w_0^2})$
- $\vec{u}^{\alpha}(x_0) =(u_0, \alpha u_0^{\alpha -1}*u'_0)$
<p> Por ejemplo, si $u(x)=3x^2-2$ tomando $x_0=1$. Cabe notar que representamos las constantes como $c=(c,0)$ y las evaluaciones como $x=(x,1)$. Llegamos entonces a que:
$$\vec{u}(x_0=1)=(3,0)*(x_0=1,1)^3-(2,0)$$
De aquí, aplicando las definiciones de las operaciones tenemos que:
$$(3,0)*(1,1)^3-(2,0)=(3,0)*(1^3,3*1^2)-(2,0)=(3,0)*(1,3)-(2,0)=(3*1,3*3+0*1)-(2,0)=(3,9)-(2,0)=(1,9)$$
Llegando finalmente a que:
$$\vec{u}(1)=(u(1),u'(1))=(1,9)$$
Por lo cual se tiene que:
$$u'(1)=9$$
Acabamos de calcular una derivada en un punto entonces sin necesidad de utilizar la definición de límites. Esta derivada es completamente exacta.

###### [1]  Construye el tipo Dual, el cual debe de tener dos campos reales.

###### [2]  Define las operaciones de suma, resta, multiplicación y división entre objetos de este tipo.

###### [3] Define las operaciones de suma, resta, multiplicación y división entre el tipo Dual y números reales. Ojo, tendrás que hacer dos definiciones, para +(Real,Dual) y para +(Dual,Real).

###### [4]  Define cómo se debe de elevar un objeto dual por una potencia con ^(Dual,Real).

###### [5] A partir de este tipo, construye una función que tome como entradas una función y un punto y te de la derivada de la función en ese punto. Para hacer esto, intenta ver qué es lo que pasa cuando evalúas tu función en un dual de la forma $(x_0,1)$. Pruébala para varias funciones y verifica su funcionamiento correcto.

###### [6] Conociendo como son las derivadas de las funciones más utilizadas, define cómo deben de operar las funciones de seno, coseno, tangente, logarítmo, exponencial y raíz cuadrada sobre un objeto de tipo dual. De nuevo, prueba tu función de derivada para varias funciones que tengan este tipo de funciones. Pueden ser tan complicadas como quieras.

###### [7] Piensa cómo podrías sacar la segunda derivada utilizando la función de derivada que acabas de programar. Ojo, vas a necesitar hacer objetos del tipo dual(dual,dual), por lo que vas a tener que hacer una definición del tipo dual con cualquier tipo de entrada, y no sólo con reales.

###### [8] Generaliza el procedimiento anterior, obteniendo una función que te saca la derivada n-ésima.